In [4]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Split Data

In [5]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                             ])

# Train Model

In [6]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [7]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

Processing
Done!
3000
750


In [8]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [66]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential()
        self.decoder = nn.Sequential()
        self.autoencoder = nn.Sequential()
        self.features = 32
        

    def forward(self, Y):
        X = Y.data
        featureSize = X.size(2) * X.size(3)
        X = Y.view(-1, featureSize)
        X = nn.Linear(featureSize, self.features)(X)
        X = F.relu(X)
        
        #Create decoder
        X = nn.Linear(self.features, featureSize)(X)
        X = F.sigmoid(X)
        X = X.view(X.data.size()[0] , X.data.size()[1])

        return X

model = AutoEncoder()

In [78]:
criterion = torch.nn.BCELoss()

In [80]:
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    output = model(data)
    loss = criterion.forward(output, data)
    print(loss)

Variable containing:
 0.7073
[torch.FloatTensor of size 1]

Variable containing:
 0.6954
[torch.FloatTensor of size 1]

Variable containing:
 0.7087
[torch.FloatTensor of size 1]

Variable containing:
 0.7077
[torch.FloatTensor of size 1]

Variable containing:
 0.6979
[torch.FloatTensor of size 1]

Variable containing:
 0.7020
[torch.FloatTensor of size 1]

Variable containing:
 0.7019
[torch.FloatTensor of size 1]

Variable containing:
 0.6991
[torch.FloatTensor of size 1]

Variable containing:
 0.7056
[torch.FloatTensor of size 1]

Variable containing:
 0.7011
[torch.FloatTensor of size 1]

Variable containing:
 0.6962
[torch.FloatTensor of size 1]

Variable containing:
 0.7057
[torch.FloatTensor of size 1]

Variable containing:
 0.6880
[torch.FloatTensor of size 1]

Variable containing:
 0.7073
[torch.FloatTensor of size 1]

Variable containing:
 0.7063
[torch.FloatTensor of size 1]

Variable containing:
 0.6984
[torch.FloatTensor of size 1]

Variable containing:
 0.6933
[torch.Floa

In [17]:
# CPU only training
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #optimizer.zero_grads()
        output = model(data)
        print(type(output))
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [ ]:
for epoch in range(1, 20):
    train(epoch)
    test(epoch, valid_loader)

# Create Sample Submission

In [59]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

In [60]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [61]:
test(1, test_loader)


Test set: Average loss: 0.1253, Accuracy: 9586/10000 (96%)



In [62]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [63]:
label_predict

array([ 7.,  2.,  1., ...,  4.,  5.,  6.])

In [64]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [65]:
predict_label.head()

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [68]:
predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)
